In [57]:
import re
import pandas as pd
import unittest
from collections import Counter
import json
import os

In [ ]:
df = pd.read_csv('../data/raw_jobs.csv')

In [2]:
def simple_regex_search(pattern:str,in_string:str):
    return re.search(pattern,in_string,re.IGNORECASE)

assert simple_regex_search('t[eé]cnic|t[eé]c.?|tec','se busca técnico en enfermería') != None

In [3]:
def multiple_patterns_match(patterns:list, in_string:str):
    return simple_regex_search("|".join(patterns),in_string)

assert multiple_patterns_match(['t[eé]cnic','t[eé]c.?','tec'],'se busca técnico en enfermería') != None

In [44]:
def find_all_matches(target_classes:dict, source_data:list):
    normalized_classes = []
    for raw_data_item in source_data:
        for target_class, patterns in target_classes.items():    
            if isinstance(patterns, dict):
                if patterns.get('Level1'):
                    result_match = find_all_matches({target_class:patterns['Level1']}, [raw_data_item])
                    if result_match: normalized_classes.append(*result_match)
                if patterns.get('Level2'):
                    patterns_first_half = patterns['Level2']['Patterns']
                    patterns_second_half = patterns['Level2']['Patterns_lvl2']
                    if multiple_patterns_match(patterns_first_half, raw_data_item):
                        result_match = find_all_matches({target_class:patterns_second_half},[raw_data_item])
                        if result_match: normalized_classes.append(*result_match)
            if isinstance(patterns, list):
                if multiple_patterns_match(patterns, raw_data_item): normalized_classes.append(target_class) 
    return normalized_classes



In [55]:
specialties_dict = {
    'Enfermería':['enfermer.?','\beu\b'],
    'Kinesiología':['kinesi.log.?','kinesiolog.?','fisioterapia','mesoterapia','erg.nomo','ergono[ií]a'],
    'TENS':{
        'Level1': [r'tens?\b'],
        'Level2': {
            'Patterns': ['t[eé]cnic','t[eé]c.?','tec'],
            'Patterns_lvl2': ['enfermer.?']
        }
    }
}
result = find_all_matches(d, 
    ['Se busca kinesiologo o técnico en enfermería para terapia intensiva', 
     ])
dict(Counter(result))

{'Enfermería': 1, 'Kinesiología': 1, 'TENS': 1}

In [54]:
with open('../normalizer/config/specialties.json') as f:
    d = json.load(f)
    print(d['Enfermería'])

['enfermer.?', '\x08eu\x08']


In [58]:
os.path.join(os.path.expanduser('~'), ".lewagon", "mlops", "data")

'/Users/pepe/.lewagon/mlops/data'

In [64]:
os.path.dirname(__file__)

TypeError: relpath() missing 1 required positional argument: 'path'

NameError: name '__file__' is not defined